In [23]:
!python -m pip install xgboost

In [24]:
from functools import partial

import os

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

import json

# plotting
import matplotlib.pyplot as plt

import importlib as imp
import sklearn_utils
imp.reload(sklearn_utils)
from sklearn_utils import run_regressor_nested_cv, run_regressor_manual

In [25]:
data = pd.read_excel("./data/dataset_pp.2022-12-09.xlsx", index_col=0)
data.columns
data["System"].unique()


array(['Mg-PR-Hh-CO', 'Mg-PR-Oh-CO', 'Mg-PR-Bh-CO', 'Mg-PR-Hv-CO',
       'Mg-PR-Ov-CO', 'Mg-PR-Bv-CO', 'Mg-PR-Hv1-CO', 'Mg-PR-Ov1-CO',
       'Mg-PR-Bv1-CO', 'Mg-MV-Hh-CO', 'Mg-MV-Bh-CO', 'Mg-MV-Hv-CO',
       'Mg-MV-Ov-CO', 'Mg-MV-Bv-CO', 'Mg-MV-Hv1-CO', 'Mg-MV-Ov1-CO',
       'Mg-MV-Bv1-CO', 'Mg-MV-Dh-CO', 'Mg-MV-Dv-CO', 'Mg-MV-Dv1-CO',
       'Mg-DV1-Hh-CO', 'Mg-DV1-Oh-CO', 'Mg-DV1-Bh-CO', 'Mg-DV1-Hv-CO',
       'Mg-DV1-Ov-CO', 'Mg-DV1-Bv-CO', 'Mg-DV1-Hv1-CO', 'Mg-DV1-Ov1-CO',
       'Mg-DV1-Bv1-CO', 'Mg-DV1-Dh-CO', 'Mg-DV1-Dv-CO', 'Mg-DV1-Dv1-CO',
       'Mg-DV2-Hh-CO', 'Mg-DV2-Oh-CO', 'Mg-DV2-Bh-CO', 'Mg-DV2-Hv-CO',
       'Mg-DV2-Ov-CO', 'Mg-DV2-Bv-CO', 'Mg-DV2-Ov1-CO', 'Mg-DV2-Bv1-CO',
       'Mg-DV2-Dh-CO', 'Mg-DV2-Dv-CO', 'Mg-DV2-Dv1-CO', 'Mg-PR-Hh-CO2',
       'Mg-PR-Oh-CO2', 'Mg-PR-Bh-CO2', 'Mg-PR-Hv-CO2', 'Mg-PR-Ov-CO2',
       'Mg-PR-Bv-CO2', 'Mg-PR-Hv1-CO2', 'Mg-PR-Ov1-CO2', 'Mg-PR-Bv1-CO2',
       'Mg-MV-Hh-CO2', 'Mg-MV-Bh-CO2', 'Mg-MV-Hv-CO2', 'Mg-MV-Ov-CO2',
       'M

In [26]:
features_primitive = data[[
    'a_lat', 'b_lat',
    'AN1', 'AN2', 'AN3', 'AN4', 'AN5', 'AN6', 'MN1', 'MN2', 'MN3', 'MN4',
    'MN5', 'MN6', 'GN1', 'GN2', 'GN3', 'GN4', 'GN5', 'GN6', 'PN1', 'PN2',
    'PN3', 'PN4', 'PN5', 'PN6', 'Elec_neg1', 'Elec_neg2', 'Elec_neg3',
    'Elec_neg4', 'Elec_neg5', 'Elec_neg6', 'At_radii1', 'At_radii2',
    'At_radii3', 'At_radii4', 'At_radii5', 'At_radii6', 'vdWR1', 'vdWR2',
    'vdWR3', 'vdWR4', 'vdWR5', 'vdWR6', 'cov_rad1', 'cov_rad2', 'cov_rad3',
    'cov_rad4', 'cov_rad5', 'cov_rad6', 'ion_rad1', 'ion_rad2', 'ion_rad3',
    'ion_rad4', 'ion_rad5', 'ion_rad6', 'desnity1', 'desnity2', 'desnity3',
    'desnity4', 'desnity5', 'desnity6']]
extra_features = data["System"].str.split("-")
extra_df = pd.DataFrame.from_dict(dict(zip(extra_features.index, extra_features.values))).T
extra_df.columns = ["...", "sys", "pos", "ads"]
extra_df = extra_df.iloc[:, 1:]
ads_names = extra_df["ads"]
for cc in ["sys", "pos", "ads"]:
    extra_df[cc] = extra_df[cc].astype("category").cat.codes
#features_primitive = pd.concat([features_primitive, extra_df], axis=1)
target = data["E_ads"]
extra_df.groupby('ads').nunique()

,sys,pos
ads,,
0,4,8
1,4,12
2,4,12


In [27]:
features_primitive

,a_lat,b_lat,AN1,AN2,AN3,AN4,AN5,AN6,MN1,MN2,...,ion_rad3,ion_rad4,ion_rad5,ion_rad6,desnity1,desnity2,desnity3,desnity4,desnity5,desnity6
0,15.221935,13.195527,12,6,8,0,0,0,24.305,12.011,...,1.40,0.00,0.00,0.00,1.738,2.26,1.4290,0.0000,0.0000,0.0000
1,15.076715,13.202952,12,6,8,0,0,0,24.305,12.011,...,1.40,0.00,0.00,0.00,1.738,2.26,1.4290,0.0000,0.0000,0.0000
2,15.179025,13.162711,12,6,8,0,0,0,24.305,12.011,...,1.40,0.00,0.00,0.00,1.738,2.26,1.4290,0.0000,0.0000,0.0000
3,15.230991,13.192220,12,6,8,0,0,0,24.305,12.011,...,1.40,0.00,0.00,0.00,1.738,2.26,1.4290,0.0000,0.0000,0.0000
4,15.258302,13.159573,12,6,8,0,0,0,24.305,12.011,...,1.40,0.00,0.00,0.00,1.738,2.26,1.4290,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,14.994697,12.012271,12,6,1,1,1,1,24.305,12.011,...,2.08,2.08,2.08,2.08,1.738,2.26,0.0899,0.0899,0.0899,0.0899
116,15.323025,12.533031,12,6,1,1,1,1,24.305,12.011,...,2.08,2.08,2.08,2.08,1.738,2.26,0.0899,0.0899,0.0899,0.0899
117,15.318144,12.565966,12,6,1,1,1,1,24.305,12.011,...,2.08,2.08,2.08,2.08,1.738,2.26,0.0899,0.0899,0.0899,0.0899
118,15.316077,12.451363,12,6,1,1,1,1,24.305,12.011,...,2.08,2.08,2.08,2.08,1.738,2.26,0.0899,0.0899,0.0899,0.0899


In [28]:
import sklearn_utils
imp.reload(sklearn_utils)
from sklearn_utils import run_regressor_nested_cv, run_regressor_manual
## FULL DATA GBR
from sklearn.ensemble import GradientBoostingRegressor


res = run_regressor_nested_cv(features_primitive, target, 
                        GradientBoostingRegressor, dict(
                            n_estimators = np.logspace(1, 2.5, num=5).astype(int),
                            max_depth = [1, 5, 6],
                            min_samples_split = [2, 3, 4]),
                        sample_class=extra_df["ads"],
                        view_class=ads_names,
                        scaler=StandardScaler, name="GB_base",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

{'train_mae': ('0.0692', '0.0253'), 'test_mae': ('0.1280', '0.0383'), 'train_mse': ('0.0158', '0.0089'), 'test_mse': ('0.0454', '0.0206'), 'train_r2': ('0.7578', '0.1428'), 'test_r2': ('0.3862', '0.2678')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [29]:
## FULL DATA GBR with higher sample-settings
from sklearn.ensemble import GradientBoostingRegressor


res = run_regressor_nested_cv(features_primitive, target, 
                        GradientBoostingRegressor, dict(
                            n_estimators = np.logspace(1, 2.5, num=10).astype(int),
                            max_depth = [1, 3, 5, 6, 8],
                            min_samples_split = [4, 6, 10],
                        min_samples_leaf = [4,5, 8]),
                        sample_class=extra_df["ads"],
                        view_class=ads_names,
                        scaler=StandardScaler, name="GB_base",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

{'train_mae': ('0.0231', '0.0179'), 'test_mae': ('0.1301', '0.0369'), 'train_mse': ('0.0021', '0.0025'), 'test_mse': ('0.0524', '0.0237'), 'train_r2': ('0.9693', '0.0367'), 'test_r2': ('0.2733', '0.3577')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [30]:
## GBR trained on 2 adsorbants, tested on another
from sklearn.ensemble import GradientBoostingRegressor


res = run_regressor_manual(
                        features_primitive[ads_names != "CO2"], target[ads_names != "CO2"],
                        features_primitive[ads_names == "CO2"], target[ads_names == "CO2"],
                        GradientBoostingRegressor, dict(
                            n_estimators = np.logspace(1, 2.5, num=5).astype(int),
                            max_depth = [1, 5, 6],
                            min_samples_split = [2, 3, 4]),
                        scaler=StandardScaler, name="GB_transfer",
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

{'train_mae': (0.1856242885709277, 0.0), 'test_mae': (0.10474875376791905, 0.0), 'train_mse': (0.0826166789804465, 0.0), 'test_mse': (0.02743184854757433, 0.0), 'train_r2': (0.14203637444328943, 0.0), 'test_r2': (-0.8566531056740774, 0.0)}


<Figure size 432x432 with 0 Axes>

In [31]:
## GBR trained on subselection
from sklearn.ensemble import GradientBoostingRegressor

selection = ads_names != "CO"
prange = [-0.3, 0.1]

res = run_regressor_nested_cv(features_primitive[selection], target[selection], 
                        GradientBoostingRegressor, dict(
                            n_estimators = np.logspace(1, 2.5, num=5).astype(int),
                            max_depth = [1, 5, 6],
                            min_samples_split = [2, 3, 4,],
                        ),
                        sample_class=ads_names[selection],
                        view_class=ads_names[selection],
                        scaler=StandardScaler, name="GB_single_class",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = prange)
                       )
print(res)

{'train_mae': ('0.0293', '0.0153'), 'test_mae': ('0.0672', '0.0121'), 'train_mse': ('0.0055', '0.0030'), 'test_mse': ('0.0193', '0.0046'), 'train_r2': ('0.4097', '0.2983'), 'test_r2': ('0.1213', '0.1712')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [32]:
from xgboost import XGBRegressor

res = run_regressor_nested_cv(features_primitive, target, 
                        partial(XGBRegressor, random_state=0), dict(
                            n_estimators = np.logspace(1, 3.5, num=5).astype(int),
                            max_depth = [0, 1, 3, 4, 5, 8],
                            learning_rate = [0.1, 0.3, 0.5, 0.7],
                            booster = ["gbtree", "gblinear"]
                        ),
                        sample_class=extra_df["ads"],
                        view_class=ads_names,
                        scaler=None, name="XGBoost",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.7, 0.7))
                       )
print(res)

{'train_mae': ('0.0274', '0.0222'), 'test_mae': ('0.1385', '0.0365'), 'train_mse': ('0.0031', '0.0036'), 'test_mse': ('0.0500', '0.0239'), 'train_r2': ('0.9551', '0.0522'), 'test_r2': ('0.3146', '0.3326')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [33]:
from sklearn.ensemble import RandomForestRegressor


res = run_regressor_nested_cv(features_primitive, target, 
                        partial(RandomForestRegressor, random_state=0), dict(
                            n_estimators = np.logspace(1, 3.5, num=5).astype(int),
                            max_depth = [None, 5, 10, 20],
                            min_samples_split = [2, 5, 10]
                        ),
                        sample_class=extra_df["ads"],
                        view_class=ads_names,
                        scaler=None, name="RF_basic",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

{'train_mae': ('0.0605', '0.0171'), 'test_mae': ('0.1168', '0.0298'), 'train_mse': ('0.0132', '0.0072'), 'test_mse': ('0.0425', '0.0190'), 'train_r2': ('0.8037', '0.1002'), 'test_r2': ('0.4128', '0.2834')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [34]:
from sklearn.kernel_ridge import KernelRidge


res = run_regressor_nested_cv(features_primitive, target, 
                        KernelRidge, dict(
                         alpha = np.logspace(-3, 2, num=5),
                         kernel= ["rbf"],
                         gamma = np.logspace(-3, 2, num=5),
                        ),
                        sample_class=extra_df["ads"],
                        view_class=ads_names,
                        scaler=None, name="KRR_basic",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

{'train_mae': ('0.0986', '0.0168'), 'test_mae': ('0.1370', '0.0238'), 'train_mse': ('0.0361', '0.0092'), 'test_mse': ('0.0626', '0.0193'), 'train_r2': ('0.4570', '0.1218'), 'test_r2': ('0.1286', '0.3129')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [35]:
# KRR WITH MORE REGULARIZATION
from sklearn.kernel_ridge import KernelRidge


res = run_regressor_nested_cv(features_primitive, target, 
                        KernelRidge, dict(
                         alpha = np.logspace(-4, 2, num=20),
                         kernel= ["rbf"],
                         gamma = np.logspace(-3, 2, num=20),
                        ),
                        sample_class=extra_df["ads"],
                        view_class=ads_names,
                        scaler=None, name="KRR_basic",
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)

{'train_mae': ('0.0900', '0.0062'), 'test_mae': ('0.1562', '0.0385'), 'train_mse': ('0.0327', '0.0037'), 'test_mse': ('0.0962', '0.0606'), 'train_r2': ('0.5058', '0.0505'), 'test_r2': ('-0.3526', '0.9062')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [36]:
# DUMMY-Baseline. Guess the median for each adsorbant!
from sklearn.dummy import DummyRegressor
from sklearn_utils import StratifiedMedianRegressor

res = run_regressor_nested_cv(extra_df[["ads"]], target, 
                        partial(StratifiedMedianRegressor), {},
                        scaler=StandardScaler, name="Dummy",
                        sample_class=extra_df["ads"],
                        view_class=ads_names,
                        test_split=0.2,
                        pp_kws = dict(
                            min_max = (-0.8, 0.8))
                       )
print(res)


{'train_mae': ('0.1371', '0.0033'), 'test_mae': ('0.1598', '0.0132'), 'train_mse': ('0.0642', '0.0026'), 'test_mse': ('0.0780', '0.0096'), 'train_r2': ('0.0279', '0.0125'), 'test_r2': ('-0.0726', '0.0693')}


<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

<Figure size 432x432 with 0 Axes>

In [37]:
np.logspace(1, 2.5, num=5).astype(int)

array([ 10,  23,  56, 133, 316])